In [ ]:
import pandas as pd
from math import sqrt
from datetime import datetime

In [ ]:
# read relevant data
obs_file = '../DATA/00_stations_in_use.csv'
obs = pd.read_csv(obs_file, delimiter=';')

dgm_file = '../DATA/DGM/dgm200_nrw.csv'
with open (dgm_file, 'r') as file_in:
    lines = file_in.readlines()
    lines = [x.strip() for x in lines]
    header = lines[0]
    del lines[0]

In [ ]:
# create header for file
n = 50
for i in range(n):
    header += ';dist_{};value_{}'.format(i+1, i+1)

In [ ]:
# function to calculate n nearest stations
def calc_nearest_stations(dgm_point, stations, number=3):
    result = []
    dgm_east = float(dgm_point[0])
    dgm_north = float(dgm_point[1])
    dgm_up = float(dgm_point[2])
    for idx, row in stations.iterrows():
        dist = sqrt((dgm_east-row['east'])**2+(dgm_north-row['north'])**2+(dgm_up-row['up'])**2)
        result.append([dist, ':{}'.format(row['name'])])
    result.sort(key=lambda x: x[0])
    result = result[0:number]
    cr = ';'.join(str(e) for e in result)
    cr = cr.replace("'", '').replace('[', '').replace(']', '').replace(', :', ';')
    return cr

In [ ]:
# calculate nearest neighbors for points of dgm, write to file
file = '../DATA/06_dgm_nearest_station-names.csv'
cc = 1
pc = 1
l = len(lines)
with open (file, 'a') as file_out: # open file to write
    file_out.write('{}\n'.format(header)) # write header
    for line in lines: # iterate dgm-file
        #compute progress
        perc = (cc/l)*100
        if perc > pc:
            print('{0}: {1:.1f}%'.format(datetime.now(), perc))
            pc+=1
        cc+=1
        # compute nearest neighbors
        dgm_point = line.split(';')
        file_out.write('{};{}\n'.format(line, calc_nearest_stations(dgm_point, obs, n)))